In [1]:
import tkinter as tk
from tkinter import filedialog as fd
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
from matplotlib.figure import Figure
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from datetime import datetime
import seaborn as sns
import os

In [8]:
areal_density = 90e15 # at/cm2 of Li
thickness_th = 68e-7 # cm ; 68 nm
li_density = areal_density * thickness_th
avo = 6.022e23 # 1/mol
li_mol = li_density / avo
print(li_mol * 1000)
print(100 * 100)

1.0162736632348058e-09
10000
